In [1]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [2]:
from pyspark.sql import functions as f
import pandas as pd

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_DayWise')\
        .getOrCreate()

22/08/25 23:02:16 WARN Utils: Your hostname, MacBook-Air-de-Willians.local resolves to a loopback address: 127.0.0.1; using 192.168.18.9 instead (on interface en0)
22/08/25 23:02:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/25 23:02:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/08/25 23:02:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/08/25 23:02:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/08/25 23:02:23 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [4]:
dataDayWise = spark.read.csv(
    'archive/day_wise.csv',
    sep = ',',
    header = True,
    )

dataDayWise.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Confirmed: string (nullable = true)
 |-- Deaths: string (nullable = true)
 |-- Recovered: string (nullable = true)
 |-- Active: string (nullable = true)
 |-- New cases: string (nullable = true)
 |-- New deaths: string (nullable = true)
 |-- New recovered: string (nullable = true)
 |-- Deaths / 100 Cases: string (nullable = true)
 |-- Recovered / 100 Cases: string (nullable = true)
 |-- Deaths / 100 Recovered: string (nullable = true)
 |-- No. of countries: string (nullable = true)



In [6]:
dataDayWise.show(6)

+----------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+----------------+
|      Date|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|No. of countries|
+----------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+----------------+
|2020-01-22|      555|    17|       28|   510|        0|         0|            0|              3.06|                 5.05|                 60.71|               6|
|2020-01-23|      654|    18|       30|   606|       99|         1|            2|              2.75|                 4.59|                  60.0|               8|
|2020-01-24|      941|    26|       36|   879|      287|         8|            6|              2.76|                 3.83|                 72.22|               9|
|2020-01-25|     1434|

In [7]:
from pyspark.sql.types import *

data_schema = [
               StructField('Date', DateType(), True),
               StructField('Confirmed', IntegerType(), True),
               StructField('Deaths', IntegerType(), True),
               StructField('Recovered', IntegerType(), True),
               StructField('Active', IntegerType(), True),
               StructField('New cases', IntegerType(), True),
               StructField('New deaths', IntegerType(), True),
               StructField('New recovered', IntegerType(), True),
               StructField('Deaths / 100 Cases', DoubleType(), True),
               StructField('Recovered / 100 Cases', DoubleType(), True),
               StructField('Deaths / 100 Recovered', DoubleType(), True),
               StructField('No. of countries', IntegerType(), True),
            ]

final_struc = StructType(fields=data_schema)

In [12]:
dataDayWise = spark.read.csv(
    'archive/day_wise.csv',
    sep = ',',
    header = True,
    schema = final_struc
    )

In [13]:
dataDayWise.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- New cases: integer (nullable = true)
 |-- New deaths: integer (nullable = true)
 |-- New recovered: integer (nullable = true)
 |-- Deaths / 100 Cases: double (nullable = true)
 |-- Recovered / 100 Cases: double (nullable = true)
 |-- Deaths / 100 Recovered: double (nullable = true)
 |-- No. of countries: integer (nullable = true)



In [14]:
dataDayWise.show(5)

+----------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+----------------+
|      Date|Confirmed|Deaths|Recovered|Active|New cases|New deaths|New recovered|Deaths / 100 Cases|Recovered / 100 Cases|Deaths / 100 Recovered|No. of countries|
+----------+---------+------+---------+------+---------+----------+-------------+------------------+---------------------+----------------------+----------------+
|2020-01-22|      555|    17|       28|   510|        0|         0|            0|              3.06|                 5.05|                 60.71|               6|
|2020-01-23|      654|    18|       30|   606|       99|         1|            2|              2.75|                 4.59|                  60.0|               8|
|2020-01-24|      941|    26|       36|   879|      287|         8|            6|              2.76|                 3.83|                 72.22|               9|
|2020-01-25|     1434|

#### Generando file parquet

In [16]:
dataDayWise.write.save('data/parquets/dataDayWise', format='parquet')